In [13]:
import os
from datetime import datetime
import numpy as np
import xarray as xr

### Download public weather dataset with user-defined spatial-temporal regions
| Weather Data | Ranges | Resolutions | Descriptions |
|-------------|---------|-------------|--------------|
| [ERA5](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=overview) |  - Spatial: Global <br>- Temporal: 1940-Present | - Spatial: ~0.25° x 0.25°<br>- Temporal: 1-Hour | Historical hourly global climate data from ECMWF reanalysis |
| [C3S](https://cds.climate.copernicus.eu/datasets/sis-energy-derived-projections?tab=overview) | - Spatial: European domain <br>- Temporal: 2005-2100| - Spatial: ~0.25° x 0.25°<br>- Temporal: 3-Hour | Projected RCP45 reference climate data from C3S Energy operational service |
<!-- | [PyPSA-Eur](https://pypsa-eur.readthedocs.io/) | Open-source dataset of European transmission network | -->


In [2]:
# Path configurations
EXTERNAL = "/Volumes/T9"  # External storage path for downloaded datasets


In [3]:
###### Define spatial-temporal region of interest ######
SPATIAL_BOUNDS = {
    'x_min': -12.0,
    'x_max': 35.0,
    'y_min': 33.0,
    'y_max': 65.0
}

In [4]:
def create_era5_dataset_slice(era_ncfile_slice):
    """Create a dataset slice with processed ERA5 data."""
    ncfile_slice = xr.Dataset(
        coords={
            'time': era_ncfile_slice.time.data,
            'y': era_ncfile_slice.latitude.data,
            'x': era_ncfile_slice.longitude.data
        },
        attrs={'module': ['era5']}
    )
    ncfile_slice = ncfile_slice.assign_coords(
        lon=('x', ncfile_slice.x.data),
        lat=('y', ncfile_slice.y.data)
    )
    
    # direct solar radiation
    ncfile_slice['influx_toa'] = xr.DataArray(
        (era_ncfile_slice['cdir']/3600).data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    ncfile_slice['influx'] = ncfile_slice['influx_toa']
    # albedo
    ncfile_slice['albedo'] = 0
    # temperature
    ncfile_slice['temperature'] = xr.DataArray(
        era_ncfile_slice['t2m'].data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    # wind speed 10 m
    ncfile_slice['wnd10m'] = xr.DataArray(
        np.sqrt(era_ncfile_slice['u10']**2 + era_ncfile_slice['v10']**2).data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    # wind direction
    wind_angle = (np.rad2deg(np.arctan2(-era_ncfile_slice['u10'], -era_ncfile_slice['v10']).data) + 360) % 360
    ncfile_slice['wnd_azimuth'] = xr.DataArray(
        wind_angle,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    # wind speed 100 m
    ncfile_slice['wnd100m'] = xr.DataArray(
        np.sqrt(era_ncfile_slice['u100']**2 + era_ncfile_slice['v100']**2).data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    if 'roughness' in era_ncfile_slice.variables:
        # surface roughness
        ncfile_slice['roughness'] = xr.DataArray(
            era_ncfile_slice['fsr'].data,
            dims=ncfile_slice.dims,
            coords=ncfile_slice.coords
        )
    else:
        ncfile_slice['roughness'] = 0
    return ncfile_slice

In [5]:
def process_era5_historical_data(year_range=[2015,2020] , month_rage=[6,9], input_path=None, output_path=None):
    """
    Process and merge ERA5 historical data for specified year range.
    
    Args:
        year_range (tuple): Start and end years for data processing
        output_path (str): Path to save processed data
    """
    nc_file_list = []
    for year in range(*year_range):
        weather_filename = os.path.join(input_path, f'weather_{year}.nc')
        ncfile = xr.open_dataset(weather_filename)
        if month_rage is not None:
            date_s = datetime(year, month_rage[0], 1, 0)
            date_e = datetime(year, month_rage[1]-1, 31, 23)
        else:
            date_s = datetime(year, 1, 1, 0)
            date_e = datetime(year, 12, 31, 23)
        era_ncfile_slice = ncfile.sel(
            time=slice(date_s, date_e),
            longitude=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
            latitude=slice(SPATIAL_BOUNDS['y_max'], SPATIAL_BOUNDS['y_min'])
        )
        era_ncfile_slice = era_ncfile_slice.isel(latitude=slice(None, None, -1))
        ncfile_slice = create_era5_dataset_slice(era_ncfile_slice)
        nc_file_list.append(ncfile_slice)
    historical_data = xr.concat(nc_file_list, dim="time")
    historical_data.to_netcdf(output_path)

In [7]:
def create_rcp45_dataset_slice(solar_ncfile_slice, 
                               wind_ncfile_slice, 
                               wind_100_ncfile_slice, 
                               tem_ncfile_slice, 
                               ave_wind_angle=None, 
                               ave_roughness=None):
    # Create dataset with all variables
    ncfile_slice = xr.Dataset(
        coords={
            'time': solar_ncfile_slice.time.data,
            'y': tem_ncfile_slice.lat.data,
            'x': tem_ncfile_slice.lon.data
        },
        attrs={'module': ['era5']}
    )
    ncfile_slice = ncfile_slice.assign_coords(
        lon=('x', ncfile_slice.x.data),
        lat=('y', ncfile_slice.y.data)
    )
    
    # Add all variables to dataset
    ncfile_slice['influx_toa'] = xr.DataArray(
        solar_ncfile_slice['rsdsAdjust'].data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    ncfile_slice['influx'] = ncfile_slice['influx_toa']
    ncfile_slice['albedo'] = 0
    ncfile_slice['temperature'] = xr.DataArray(
        tem_ncfile_slice['tasAdjust'].data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    ncfile_slice['wnd10m'] = xr.DataArray(
        wind_ncfile_slice['sfcWindAdjust'].data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    ncfile_slice['wnd100m'] = xr.DataArray(
        wind_100_ncfile_slice['ws100'].data,
        dims=ncfile_slice.dims,
        coords=ncfile_slice.coords
    )
    if ave_wind_angle is not None:
        ncfile_slice['wnd_azimuth'] = xr.DataArray(
            ave_wind_angle,
            dims=ncfile_slice.dims,
            coords=ncfile_slice.coords
            )
    if ave_roughness is not None:
        ncfile_slice['roughness'] = xr.DataArray(
            ave_roughness,
            dims=ncfile_slice.dims,
            coords=ncfile_slice.coords
        )
    return ncfile_slice

In [8]:
def process_rcp45_data(year_range=[2015,2020] , month_rage=[6,9], input_path=None, output_path=None):
    nc_file_list = []
    rcp = 45
    for year in range(*year_range):
        date_s = datetime(year, month_rage[0], 1, 0)
        date_e = datetime(year, month_rage[1]-1, 31, 23)
        # Process solar radiation data
        weather_filename = input_path + f'/rsdsAdjust_EUR-25_MPI-M-MPI-ESM-LR_rcp{rcp}_r1i1p1_CLMcom-CCLM4-8-17_v1-IPSL-CDFT22-ERA5-1980-2018_3hr_195001010130-210012312230.nc'
        solar_ncfile = xr.open_dataset(weather_filename, mode='r')
        solar_ncfile_slice = solar_ncfile.sel(
            time=slice(date_s, date_e),
            lon=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
            lat=slice(SPATIAL_BOUNDS['y_min'], SPATIAL_BOUNDS['y_max'])
        )
        
        # Process wind speed at 10m data
        weather_filename = input_path + f'/sfcWindAdjust_EUR-25_MPI-M-MPI-ESM-LR_rcp{rcp}_r1i1p1_CLMcom-CCLM4-8-17_v1-IPSL-CDFT22-ERA5-1980-2018_3hr_1950010100-2100123121.nc'
        wind_ncfile = xr.open_dataset(weather_filename, mode='r')
        wind_ncfile_slice = wind_ncfile.sel(
            time=slice(date_s, date_e),
            lon=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
            lat=slice(SPATIAL_BOUNDS['y_min'], SPATIAL_BOUNDS['y_max'])
        )
        
        # Process wind speed at 100m data
        weather_filename = input_path + f'/ws100_EUR-25_MPI-M-MPI-ESM-LR_rcp{rcp}_r1i1p1_CLMcom-CCLM4-8-17_v1-IPSL-CDFT22-ERA5-1980-2018_3hr_195001010000-210012312100.nc'
        wind_100_ncfile = xr.open_dataset(weather_filename, mode='r')
        wind_100_ncfile_slice = wind_100_ncfile.sel(
            time=slice(date_s, date_e),
            lon=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
            lat=slice(SPATIAL_BOUNDS['y_min'], SPATIAL_BOUNDS['y_max'])
        )
        
        # Process temperature data
        weather_filename = input_path + f'/tasAdjust_EUR-25_MPI-M-MPI-ESM-LR_rcp{rcp}_r1i1p1_CLMcom-CCLM4-8-17_v1-IPSL-CDFT22-ERA5-1980-2018_3hr_1950010100-2100123121.nc'
        tem_ncfile = xr.open_dataset(weather_filename, mode='r')
        tem_ncfile_slice = tem_ncfile.sel(
            time=slice(date_s, date_e),
            lon=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
            lat=slice(SPATIAL_BOUNDS['y_min'], SPATIAL_BOUNDS['y_max'])
        )

        """
        wind_angle and roughness: no prediction dataset available, 
        use three hour average as the future data projection historical average 
        """
        ave_u10 = []
        ave_v10 = []
        ave_roughness = []
        for year in range(2019, 2024):
            weather_filename = EXTERNAL + f'/era5/heatwave_july/weather_{year}.nc'
            ncfile = xr.open_dataset(weather_filename)
            date_s = datetime(year, month_rage[0], 1, 0)
            date_e = datetime(year, month_rage[1]-1, 31, 23)
            era_ncfile_slice = ncfile.sel(
                time=slice(date_s, date_e),
                longitude=slice(SPATIAL_BOUNDS['x_min'], SPATIAL_BOUNDS['x_max']),
                latitude=slice(SPATIAL_BOUNDS['y_max'], SPATIAL_BOUNDS['y_min'])
            ) #(note era5 data has reverse lat)
            era_ncfile_slice = era_ncfile_slice.isel(latitude=slice(None, None, -1))
            ave_u10.append(era_ncfile_slice['u10'])
            ave_v10.append(era_ncfile_slice['v10'])
            ave_roughness.append(era_ncfile_slice['fsr'])
            
        ave_u10 = np.stack(ave_u10, axis=0).mean(0)
        ave_v10 = np.stack(ave_v10, axis=0).mean(0)
        ave_u10 = ave_u10.reshape(-1,3,ave_u10.shape[1],ave_u10.shape[2]).mean(1)
        ave_v10 = ave_v10.reshape(-1,3,ave_u10.shape[1],ave_u10.shape[2]).mean(1)
        ave_wind_angle = (np.rad2deg(np.arctan2(-ave_u10, -ave_v10)) + 360)%360
        ave_roughness = np.stack(ave_roughness, axis=0).mean(0)
        ave_roughness = ave_roughness.reshape(-1,3,ave_u10.shape[1],ave_u10.shape[2]).mean(1)

        # Create dataset with all variables
        ncfile_slice = create_rcp45_dataset_slice(solar_ncfile_slice, 
                                                  wind_ncfile_slice, 
                                                  wind_100_ncfile_slice, 
                                                  tem_ncfile_slice, 
                                                  ave_wind_angle, 
                                                  ave_roughness)
        nc_file_list.append(ncfile_slice)
    
    # Concatenate all years and save to netCDF
    future_data = xr.concat(nc_file_list, dim="time")
    future_data.to_netcdf(output_path)

In [ ]:
"""
#  Process historical ERA5 data (2015-2019 for demand calibration)
"""
# Input_path = os.path.join(EXTERNAL, "era5/demand_cali")
# Output_path = os.path.join(EXTERNAL, "era5/demand_cali/Historical_era5_data.nc")
process_era5_historical_data((2015, 2020), 
                             None, 
                             EXTERNAL + f"/era5/demand_cali", 
                             EXTERNAL + f"/era5/demand_cali/Historical_era5_data.nc")

In [ ]:
"""
#  Process historical ERA5 data (2019-2024, Jun., Jul., Aug. for heatwave generation)
"""
# Input_path = os.path.join(EXTERNAL, "era5/heatwave_july")
# Output_path = os.path.join(EXTERNAL, "era5/heatwave_july/Historical_era5_July_data.nc")
process_era5_historical_data((2019, 2024), 
                             (6,9), 
                             EXTERNAL + f"/era5/heatwave_july", 
                             EXTERNAL + f"/era5/heatwave_july/Historical_era5_July_data.nc")

In [ ]:
"""
# Process historical RCP45 data (2019-2024, Jun., Jul., Aug.) for heatwave calculation
"""
# Input_path = EXTERNAL + f"/rcp45"
# Output_path= EXTERNAL + f"/rcp45/Historical_rcp45_July_data.nc"
process_rcp45_data((2019,2024), 
                   (6,9), 
                   EXTERNAL + f"/rcp45", 
                   EXTERNAL + f"/rcp45/Historical_rcp45_July_data.nc")

In [11]:
"""
# Process future RCP45 data (2025-2030, Jun., Jul., Aug.) for heatwave generation
"""
# Input_path = EXTERNAL + f"/rcp45"
# Output_path= EXTERNAL + f"/rcp45/Future_rcp45_July_data.nc"
process_rcp45_data((2025,2031), 
                   (6,9), 
                   EXTERNAL + f"/rcp45", 
                   EXTERNAL + f"/rcp45/Future_rcp45_July_data.nc")

In [12]:
# his_era5_ncfile = xr.open_dataset(EXTERNAL + f"/era5/demand_cali/Historical_era5_data.nc")
his_era5_ncfile = xr.open_dataset(EXTERNAL + f"/era5/heatwave_july/Historical_era5_July_data.nc")
his_rcp45_ncfile = xr.open_dataset(EXTERNAL + f"/rcp45/Historical_rcp45_July_data.nc")
fut_rcp45_ncfile = xr.open_dataset(EXTERNAL + f"/rcp45/Future_rcp45_July_data.nc")